## 방법

1. 카카오톡 대화방에서 가장 친한 친구 또는 가장 많이 대화한 친구 1명 대화 내용을 다운로드 받아주세요.
    * 윈도우 pc 기준으로 [대화방] -> [설정] -> [대화 내용] -> [대화 내보내기] 단계로 다운로드 받아주세요.
    * mac에서 금지 (저장되는 형식이 다름)
2. 저장된 친구 이름으로 chat 폴더에 txt 또는 csv를 저장해주세요. (ex) 성록교수님짱짱.txt
3. 친구의 카톡 이름을 입력하여주세요.
4. 자신의 카톡 이름을 입력하여주세요.
5. 평소에 자신이 욕 대신 사용하는 단어를 적어주세요. (ex) 시발 => 시방
    * 한 단어씩 입력하고 enter를 눌러주세요.
    * 더 이상 없는 경우 '없음'을 작성하고 enter를 눌러주세요.
6. 배경 이미지 같은 경우에는 선택사항입니다.
    * 배경색이 흰색 혹은 투명색인 이미지가 정확하게 원하는 모양을 반영하기에 좋습니다.
    * 만약 있다면 Image 폴더에 저장하고 jpg 또는 png를 빼고 이름만 입력하여 주세요.


## 주의사항

1. 개인정보 보호 처리가 되어 있지 않습니다.
    * 주소, 계좌, 비밀번호 등 유출되지 않게 조심하세요.
    * 어떠한 데이터도 수집되지 않음.
2. hanspell 라이브러리를 사용하여 오타가 아닌 부분이 자동으로 수정되어 있을 수 있습니다.
    * 주로 사람 이름을 불렀을 경우 수정될 수 있습니다.
    * hanspell 라이브러리에 예외 사항으로 걸리는 경우 맞춤법 처리가 되지 않아 오류 데이터가 될 수 있습니다.
3. 너무 긴 대화는 시간이 매우 오래 걸립니다.
    * 긴 파일들은 10000개의 대화 기준으로 자르도록 하겠습니다. 10000개 기준 약 340초~400초 소요됨
4. 맥을 사용하지 않는다면 font의 path를 따로 설정해주세요.

In [1]:
# 모든 import

import time
import csv
from hanspell import spell_checker

from khaiii import KhaiiiApi
from konlpy.tag import Twitter, Kkma, Komoran

from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np

from gensim.models import Word2Vec

KeyboardInterrupt: 

In [ ]:
def all_input():
    os = input('다운로드 환경이 window이면 1번을, mac이면 2번을 입력하여 주세요 :')
    
    friend_name = input('친구의 카톡 닉네임을 입력하여 주세요 : ')

    my_name = input('자신의 카톡 닉네임을 입력하여 주세요 : ')
    
    line = set_file(os,friend_name,my_name)
    
    print("평소에 사용하는 비속어를 적어주세요. 없을 경우 '없음'을 입력하여 주세요.")
    
    while True:
        temp = input()
        if temp == '없음':
            break
        input_explicits.append(temp)
    set_explicits()
        
    print('원하는 배경 이미지가 있다면, 파일 이름을 입력하여 주세요. 없을 경우 "없음"을 입력하여 주세요.')
    mask_arr = input('이미지 파일 이름 입력(.png 파일) :'  )

    if mask_arr != '없음':
        im = Image.open(f'./Image/{mask_arr}.png') # 이미지 파일 읽어오기
        mask_arr = np.array(im) # 픽셀 값 배열 형태 변환
    
    return os,friend_name,my_name,mask_arr

In [ ]:
def set_file(os,f_name,m_name):
    lines = ''
    line = []
    
    if os == '1':
        file = './chat/'+f_name+'.txt'
        with open(file, 'r', encoding="utf-8") as f:
            lines = f.readlines()
        linelist = [line for line in lines if '] [' in line]
        for value in linelist:
            value = value.split(']')
            name=value[0]
            line.append([name[1:], value[2]])
        
    elif os == '2':
        file = './chat/'+f_name+'.csv'
        with open(file,newline='') as cf:
            cr = csv.reader(cf)
            for word in cr:
                lines += str(word[2])
                lines += ' '
                line.append([word[1],word[2]])
    else:
        print('프로그램을 다시 실행하여 주세요.')
        
    return line,lines

In [ ]:
def replace_value(value):
    value = value.replace('사진\n', '').replace('이모티콘\n','').replace(
                '삭제된 메세지입니다', '').replace('ㅋ','').replace('ㅠ','').replace('ㅌ','').replace('ㅎ','')
    return value

In [ ]:
def preProcess_line(text):
    
    start = time.time() 

    # 카톡 대화 내용이 많을 경우 제한사항
    if len(text) > 10000 :
        text = text[:10000]
    
    for value in text:
        name = value[0]
        sentence = value[1]
        sentence = replace_value(sentence)

        # 특수문자 제거
        sentence = ''.join(char for char in sentence if char.isalnum())

        if sentence == '':
            continue

        try:
            sentence = spell_checker.check(sentence).checked
            
        except Exception as e:
            print('예외 발생', value)
        

        finally:
            all_text.append(sentence)
            if name == friend_name:
                friend_text.append(sentence)
            elif name == my_name:
                my_text.append(sentence)
            else:
                print("친구 이름 또는 본인의 이름을 정확히 입력하여 주세요.")

    spend_time = time.time() - start
    return all_text,friend_text,my_text

In [ ]:
# 카카오톡 khaiii 사용해보기

def kakao_khaiii(text,name):
    text_analyzed = []
    api = KhaiiiApi()

    for sentence in text:
        analyzed = api.analyze(sentence)
        for word in analyzed:
            for morph in word.morphs:
                if morph.tag == 'NNP'or morph.tag == 'NNG':
                    if len(morph.lex) > 1:
                        text_analyzed.append(morph.lex)

    counts = Counter(text_analyzed)
    tags = counts.most_common(40)
    text_analyzed = set(text_analyzed)

    make_cloud(dict(tags),name,'khaiii')

In [ ]:
# khaiii를 통한 WordCloud 생성

def make_cloud(dict_text,name,text):
    if mask_arr == "없음":
        wc = WordCloud(font_path="/System/Library/Fonts/Supplemental/AppleGothic.ttf",background_color="white",
                       width = 700, height = 700, colormap = 'autumn', prefer_horizontal = True).generate_from_frequencies(dict_text)
    else :
        wc = WordCloud(font_path="/System/Library/Fonts/Supplemental/AppleGothic.ttf",background_color="white",
                   width = 700, height = 700, colormap = 'autumn', mask = mask_arr, prefer_horizontal = True).generate_from_frequencies(dict_text)
            
    plt.rcParams['font.family'] ='Malgun Gothic'
    plt.rcParams['axes.unicode_minus'] =False
    plt.figure(figsize = (6, 6))  
    plt.imshow(wc)
    plt.title(f"{name}'s Word Frequency by {text}", size = 13)

    plt.axis('off')

    plt.show()
            

In [ ]:
# Kkma를 사용해보기

def  use_Komoran(text,name):
    text_analyzed_Komoran = []
    komoran = Komoran()

    for sentence in text:
        text_analyzed_Komoran.extend(komoran.nouns(sentence))

    text_analyzed_Komoran = [n for n in text_analyzed_Komoran if len(n) > 1]
    counts= Counter(text_analyzed_Komoran)
    tags = counts.most_common(40)
    
    make_cloud(dict(tags),name,'Komoran')

In [ ]:
def set_explicits():
    explicit_lines = ''
    my_explicits_dict = {}
    with open('my_dictionary.txt', 'r', encoding="utf-8") as f:
        explicit_lines = f.readlines()

    for value in explicit_lines:
        my_explicits.append(value[:-6])

    my_explicits.extend(input_explicits)

In [ ]:
# 비속어 검사

def count_explicits(line,name):
    explicits_dict = {}
    for value in line:
        value = value.split(' ')
        for word in value:
            if word in my_explicits:
                if word in explicits_dict:
                    explicits_dict[word] += 1
                else :
                    explicits_dict[word] = 1
    
    return explicits_dict

In [ ]:
def analyzed_chat_for_fun(all_text):
    komoran = Komoran(userdic='my_dictionary.txt')
    tokenizedData = []

    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    for sent in all_text:
        t = (komoran.nouns(sent))
        stopwords_removed_t = [w for w in t if w not in stopwords]
        stopwords_removed_t = [ w for w in stopwords_removed_t if len(w)>1]
        tokenizedData.append(stopwords_removed_t)

    tokenizedData = tokenizedData[1:]
    
    used_explicits = []
    used_explicits.extend(list(count_explicits(my_text,my_name).keys()))
    used_explicits.extend(list(count_explicits(friend_text,friend_name).keys()))

    used_explicits = set(used_explicits)
    used_explicits = list(used_explicits)
    
    model = Word2Vec(sentences = tokenizedData, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

    for value in used_explicits:
        try:
            print(f"주로 '{model.wv.most_similar(value)[0][0]}'를 말하며 비속어를 사용하는 것으로 보입니다")
        except:
            pass
    
    count_my_explicits = count_explicits(my_text,my_name)
    count_friend_explicits = count_explicits(friend_text,friend_name)

    print(f'다음은 {my_name}님이 사용하신 욕입니다 : {count_my_explicits}')
    print(f'다음은 {friend_name}님이 사용하신 욕입니다 : {count_friend_explicits}')

    my_all_counts = 0
    friend_all_counts = 0

    for i in list(count_my_explicits.keys()):
        my_all_counts+=count_my_explicits[i]

    for j in list(count_friend_explicits.keys()):
        friend_all_counts += count_friend_explicits[j]
    
    if my_all_counts>friend_all_counts:
        print(f'{my_name}님이 {friend_name}님 보다 사용하신 비속어 횟수가 {my_all_counts-friend_all_counts}회만큼 더 많습니다.')

    elif friend_all_counts>my_all_counts:
        print(f'{friend_name}님이 {my_name}님 보다 사용하신 비속어 횟수가 {friend_all_counts-my_all_counts}회만큼 더 많습니다.')

    else:
        print(f'{friend_name}님과 {my_name}님이 사용하신 비속어 횟수가 {friend_all_counts}회로 동일합니다.')
        
    print(f'욕설 1회당 벌금 1000원을 납부하신다면, {(friend_all_counts+my_all_counts)*1000}원만큼 모으실 수 있었습니다.')
    print("연구 결과에 따르면 나쁜 습관을 고치는 데 21일이 걸린다고 합니다다.")
    print("21일 동안 벌금 0원 모으기에 도전하세요!")

In [ ]:
def print_friend_result():
    kakao_khaiii(friend_text,friend_name)
    use_Komoran(friend_text,friend_name)

def print_my_result():
    kakao_khaiii(my_text,my_name)
    use_Komoran(my_text,my_name)

    
def print_all_result():
    kakao_khaiii(all_text,'우리')
    use_Komoran(all_text,'우리')
    count_explicits(all_text,'우리')
    analyzed_chat_for_fun(all_text)

# 모든 입력

1. 카카오톡 파일 다운로드 환경
2. 친구 카톡 이름
3. 자신의 카톡 이름
4. 평소에 자기가 사용하는 비속어
5. 배경 이미지

In [ ]:
# 주요 변수들 모음

friend_name = ''
my_name = ''
file = ''
os = ''

my_explicits = []
input_explicits = []

spend_time = '' # 전체 대화 내용 소요 된 시간
mask_arr = '' #wordCloud 배경 이미지

lines = '' # 친구와의 카톡에 모든 대화 (전처리 과정 없음)
line = [] # 대화 내용을 포함한 라인의 list (년도 미포함, 부가 정보 미포함)

all_text = [] # 전처리 과정 이후 카톡의 모든 대화
my_text = [] # 전처리 과정 이후 내가 보낸 대화
friend_text = [] # 전처리 과정 이후 친구가 보낸 대화

os,friend_name,my_name,mask_arr = all_input()
line,lines = set_file(os,friend_name,my_name)
all_text,friend_text,my_text = preProcess_line(line)

## 모든 결과 출력하기
1. 친구 wordCloud by khaiii
2. 친구 wordCloud by Komoran
3. 나의 wordCloud by khaiii
4. 나의 wordCloud by Komoran
5. 우리의 wordCloud by khaiii
6. 우리의 wordCloud by Komoran
7. 우리의 욕설 및 비속어 사용 분석

In [ ]:
print_friend_result()

In [ ]:
print_my_result()

In [ ]:
print_all_result()